## 4.5 PUMApp

W tej aplikacji wykorzystamy `RecyclerView`, `Drawer`, `Jetpack Navigation` oraz `Materials` do stworzenia prostej aplikacji będącej mocno uproszczoną wersją tego repozytorium.

### **Fragmenty**

Rozpocznijmy od dodania fragmentów (**Blank Fragment**) z których będzie składała się nasza aplikacja.

- `StartFragment` - fragment domowy
- `ModuleListFragment` - fragment zawierający listę modułów
- `ModuleFragment` - fragment zawierający zawartość modułu
- `LectureFragment` - fragment zawierający informacje o wykładzie
- `LabListFragment` - fragment zawierający listę wszystkich labów przeznaczonych do tego laboratorium
- `AppListFragment` - fragment zawierający listę wszystkich aplikacji przeznaczonych do tego laboratorium
- `AboutFragment` - fragment zawierający podstawowe informacje o aplikacji
- `SettingsFragment` - fragment zawierający ustawienia aplikacji - w tej aplikacji dodamy możliwość przełączenia na ciemny motyw

### **Nawigacja**

Dodajmy do aplikacji nawigację. Do skryptów `Gradle(Project)` dodajemy

In [ ]:
buildscript {
    repositories {
        google()
    }
    dependencies {
        def nav_version = "2.5.0"
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:$nav_version"
    }
}

Do `Gradle(Module)`

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
}

dependencies {

    def nav_version = "2.5.0"
    implementation "androidx.navigation:navigation-fragment:$nav_version"
    implementation "androidx.navigation:navigation-ui:$nav_version"
}

Następnie synchronizujemy projekt i przechodzimy do utworzenia grafu.

<img src="https://files.fm/thumb_show.php?i=375bnxj9c" width="400" />

Zauważmy że `AboutFragment` oraz `SettingsFragment` nie posiadają żadnej zdefiniowanej akcji - nawigację do tych fragmentów dodamy za pomocą `Drawer`.

Dodajmy `Menu` dla `Drawer`. Otwieramy menu kontekstowe na katalogu `res`, następnie **New -> Android Resource File**, jako **Resource Type** wybieram `Menu` - plik nazwijmy `drawer_menu`. Dodamy w nim dwa wpisy `<item>`

```xml
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@+id/aboutFragment"
        android:title="About" />

    <item
        android:id="@+id/settingsFragment"
        android:title="Settings" />
</menu>
```

Istotną rzeczą są `id` - muszą być takie same jak w `navigation.xml` pola `id` nadane odpowiednim fragmentom. Tutaj chcemy zapewnić możliwość nawigacji z menu bocznego do dwóch fragmentów, więc dodajemy dwa pola o odpowiednich `id`.

Kolejnym krokiem będzie modyfikacja layoutu `MainActivity`. Naszym głównym elementem będzie `DrawerLayout` - ponieważ chcemy móc otworzyć panel boczny, tutaj będzie on powiązany z aktywnością główną aplikacji.

```xml
<androidx.drawerlayout.widget.DrawerLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:id="@+id/drawerLayoutID"
    tools:context=".MainActivity">
```

Następnie potrzebujemy `FragmentContainer` - jak w poprzednich aplikacjach

```xml
    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:orientation="vertical">

        <androidx.fragment.app.FragmentContainerView
            android:id="@+id/nav_host_fragment"
            android:name="androidx.navigation.fragment.NavHostFragment"
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            app:defaultNavHost="true"
            app:navGraph="@navigation/navigation" />
    </LinearLayout>
```

Ostatnim elementem jest `NavigationView`, który pozwoli dodać wcześniej utworzony `drawer_menu` i wyświetlić panel boczny

```xml
    <com.google.android.material.navigation.NavigationView
        android:id="@+id/navigation_drawerID"
        android:layout_width="wrap_content"
        android:layout_height="match_parent"
        android:layout_gravity="start"
        app:menu="@menu/drawer_menu"/>

</androidx.drawerlayout.widget.DrawerLayout>
```



Ostatnim krokiem w tworzeniu nawigacji w tym projekcie będzie spięcie wszystkiego w `MainActivity`. W pierwszym kroku musimy posiadać instancję `NavController`

In [ ]:
private val navController: NavController by lazy {
    val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
            as NavHostFragment
    navHostFragment.findNavController()
}

Następnie definiujemy `AppBarConfiguration` w którym łączymy `NavController` z `DrawerLayout`

In [ ]:
private val appBarConfiguration: AppBarConfiguration by lazy {
    val drawerLayout : DrawerLayout = findViewById(R.id.drawerLayoutID)
    AppBarConfiguration(navController.graph, drawerLayout)
}

W metodzie `onCreate` łączymy domyślny `ActionBar` z `NavController`

In [ ]:
setupActionBarWithNavController(navController, appBarConfiguration)

oraz `NavigationView` z `NavController`

In [ ]:
findViewById<NavigationView>(R.id.navigation_drawerID).setupWithNavController(navController)

Dodamy implementacje dwóch metod. Pierwszą jest `onSupportNavigateUp` - pozwala ona na zmianę ikony szuflady na ikonę powrotu po opuszczeniu głównego ekranu aplikacji - szufladę zawsze można otworzyć przez przeciągnięcie.

In [ ]:
override fun onSupportNavigateUp(): Boolean {
    return navController.navigateUp(appBarConfiguration)
            || super.onSupportNavigateUp()
}

Następną metodą jest `onOptionsItemSelected` - pozwala ona na nawigację bez definiowania akcji pod warunkiem zgodności `id` w plikach `drawer_menu` oraz `navigation`

In [ ]:
override fun onSupportNavigateUp(): Boolean {
    return navController.navigateUp(appBarConfiguration)
            || super.onSupportNavigateUp()
}

Możemy przetestować działanie `Drawer`

<img src="https://media1.giphy.com/media/v98tLyiUpSyfTG9VmD/giphy.gif?cid=790b76110cf80882ea0dfcb8fdb4c8c397f8a3d45e0401df&rid=giphy.gif&ct=g" width="150" />

### **Zmiana motywu**

Dodajmy opcję zmiany motywu w naszej aplikacji - wykorzystamy w tym celu `RadioButton` oraz `RadioGroup`. Dodamy możliwość wyboru trzech motywów - jasny, ciemny, domyślny zintegrowany z Android. Możemy zauważyć że dwa motywy mamy już domyślnie zdefiniowane w plikach `themes.xml`. Rozpocznijmy od layoutu `fragment_settings`

```xml
<?xml version="1.0" encoding="utf-8"?>
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.SettingsFragment">

    <RelativeLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent">

        <TextView
            android:id="@+id/textViewSettings"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginTop="30dp"
            android:gravity="center_horizontal"
            android:text="@string/default_value"
            android:textAlignment="center"
            android:textSize="36sp" />

        <TextView
            android:id="@+id/textViewThemeSetting"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_below="@id/textViewSettings"
            android:layout_marginTop="30dp"
            android:gravity="center_horizontal"
            android:text="@string/theme_settings"
            android:textAlignment="center"
            android:textSize="28sp" />

        <TextView
            android:id="@+id/textViewCurrentTheme"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_below="@id/textViewThemeSetting"
            android:layout_marginTop="6dp"
            android:gravity="center_horizontal"
            android:text="@string/current_theme"
            android:textAlignment="center"
            android:textSize="20sp" />

        <RadioGroup
            android:id="@+id/radioButtonTheme"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_below="@id/textViewCurrentTheme"
            android:layout_marginTop="30dp"
            android:orientation="vertical"
            android:layout_marginStart="100dp"
            android:layout_centerHorizontal="true"
            android:padding="4dp">

            <RadioButton
                android:id="@+id/radioButtonLight"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:layout_margin="5dp"
                android:padding="5dp"
                android:text="@string/light"
                android:textSize="24sp"/>

            <RadioButton
                android:id="@+id/radioButtonDark"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:layout_margin="5dp"
                android:layout_weight="1"
                android:padding="5dp"
                android:text="@string/dark"
                android:textSize="24sp"/>

            <RadioButton
                android:id="@+id/radioButtonDefault"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:layout_margin="5dp"
                android:layout_weight="1"
                android:padding="5dp"
                android:text="@string/system_default"
                android:textSize="20sp"/>

        </RadioGroup>
    </RelativeLayout>

</FrameLayout>
```

dodajmy odpowiednie elementy do `strings`

```xml
<resources>
    <string name="app_name">PUMApp</string>
    <!-- TODO: Remove or change this placeholder text -->
    <string name="hello_blank_fragment">Hello blank fragment</string>
    <string name="settings">Settings</string>
    <string name="default_value">Settings</string>
    <string name="light">Light</string>
    <string name="dark">Dark</string>
    <string name="light_theme">Light Theme</string>
    <string name="dark_theme">Dark Theme</string>
    <string name="system_default">Android Compatible Theme</string>
    <string name="selected_language">English</string>
    <string name="theme_settings">Theme Settings</string>
    <string name="current_theme">Default</string>
</resources>
```

Przejdźmy do `SettingsFragment` i zmieńmy tytuł na `ActionBar`. W metodzie `onCreateView` muisimy się dostać do `ActionBar`, w tym celu potrzebujemy instancję `AppCompatActivity` na której możemy wywołać metodę `getSupportActionBar`.

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View? {
    val activity: AppCompatActivity = activity as AppCompatActivity
    activity.supportActionBar?.title = getString(R.string.settings)
    return inflater.inflate(R.layout.fragment_settings, container, false)
}

Zmianę motywu wykonujemy przez wywołanie metody `AppCompatDelegate.setDefaultNightMode` z odpowiednimi opcjami.

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)

    val themeTextView: TextView = view.findViewById(R.id.textViewCurrentTheme)
    view.findViewById<RadioGroup>(R.id.radioButtonTheme).setOnCheckedChangeListener { _, checkedId ->
        when(checkedId){
            R.id.radioButtonLight -> {
                themeTextView.text = getString(R.string.light_theme)
                AppCompatDelegate.setDefaultNightMode(AppCompatDelegate.MODE_NIGHT_NO)
            }
            R.id.radioButtonDark -> {
                themeTextView.text = getString(R.string.dark_theme)
                AppCompatDelegate.setDefaultNightMode(AppCompatDelegate.MODE_NIGHT_YES)
            }
            R.id.radioButtonDefault -> {
                themeTextView.text = getString(R.string.current_theme)
                AppCompatDelegate.setDefaultNightMode(AppCompatDelegate.MODE_NIGHT_FOLLOW_SYSTEM)
            }
        }
    }
}

<img src="https://media3.giphy.com/media/xY5ZRDy5mPHiMIzGwH/giphy.gif?cid=790b7611e9194faebe71b6ed8cb1782db66bd3838afe9a77&rid=giphy.gif&ct=g" width="150" />

Zmodyfikujmy nieco wygląd szuflady. Po pierwsze dodamy nagłówek. Do katalogu `layout` dodajemy plik `drawer_header`

```xml
<?xml version="1.0" encoding="utf-8"?>
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="200dp"
    android:orientation="horizontal"
    android:background="?colorSecondary"
    android:gravity="center">

    <ImageView
        android:id="@+id/android_drawer_imageView"
        android:layout_width="wrap_content"
        android:layout_height="match_parent"
        android:src="@drawable/ic_android"
        android:contentDescription="@string/android" />

    <TextView
        android:layout_width="wrap_content"
        android:layout_height="match_parent"
        android:text="@string/pum"
        android:textSize="72sp"
        android:gravity="center"
        android:layout_toEndOf="@id/android_drawer_imageView"
        android:textColor="?colorOnSecondary"/>



</RelativeLayout>
```

Do kolorów zdefiniowanych w `themes.xml` odnosimy się poprzez `?nazwa_atrybutu`

```xml
android:background="?colorSecondary"
```

Grafikę definiujemy w katalogu `drawable` - otwieramy menu kontekstowe i wybieramy **New -> Vector Asset**, po kliknięciu na **ClipArt** możemy wybrać odpowiedni wygląd, lub zdefiniować własny

```xml
<vector android:height="76dp" android:tint="#000000"
    android:viewportHeight="24" android:viewportWidth="24"
    android:width="76dp" xmlns:android="http://schemas.android.com/apk/res/android">
    <path android:fillColor="@android:color/white" android:pathData="M16,1L8,1C6.34,1 5,2.34 5,4v16c0,1.66 1.34,3 3,3h8c1.66,0 3,-1.34 3,-3L19,4c0,-1.66 -1.34,-3 -3,-3zM14,21h-4v-1h4v1zM17.25,18L6.75,18L6.75,4h10.5v14z"/>
</vector>
```

Dodajmy również separator z nazwą oraz grafiki do `drawer_menu`

```xml
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item android:title="@string/settings">
        <menu>
            <item
                android:id="@+id/aboutFragment"
                android:icon="@drawable/ic_about"
                android:title="@string/about" />

            <item
                android:id="@+id/settingsFragment"
                android:icon="@drawable/ic_settings"
                android:title="@string/settings" />

        </menu>
    </item>
</menu>
```

Aby wykorzystać utworzony nagłówek musimy dodać go do `NavigationView` w pliku `activity_main`

```xml
app:headerLayout="@layout/drawer_header"

    <com.google.android.material.navigation.NavigationView
        android:id="@+id/navigation_drawerID"
        android:layout_width="wrap_content"
        android:layout_height="match_parent"
        android:layout_gravity="start"
        app:headerLayout="@layout/drawer_header"
        app:menu="@menu/drawer_menu"/>
```

<img src="https://media4.giphy.com/media/GUtkxcigj4yDoiozTP/giphy.gif?cid=790b7611d67892dbf4b580d998ea69d1e48b7abe07bfcc04&rid=giphy.gif&ct=g" width="150" />

Dodajmy następnie przycisk do `StartFragment` w taki sam sposób jak w `QuickYogaApp`

`start_button.xml` do katalogu `drawable`

```xml
<?xml version="1.0" encoding="utf-8"?>
<shape xmlns:android="http://schemas.android.com/apk/res/android"
    android:shape="rectangle">
    <stroke android:width="7dp"
        android:color="?colorSecondary"/>
    <solid android:color="?colorSecondaryVariant"/>
    <corners android:radius="170dp"/>

</shape>
```

`start_button_ripple.xml` do katalogu `drawable`

```xml
<?xml version="1.0" encoding="utf-8"?>
<ripple xmlns:android="http://schemas.android.com/apk/res/android"
    android:color="?colorSecondary">
    <item android:id="@+id/ripple_effect">
        <shape android:shape="rectangle"/>
    </item>
    <item android:drawable="@drawable/start_button"/>

</ripple>
```

zmodyfikujmy layout `StartFragment`

```xml
<?xml version="1.0" encoding="utf-8"?>
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.StartFragment">

    <TextView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="@string/pum"
        android:textSize="124sp"
        android:layout_gravity="center_horizontal"
        android:layout_marginTop="36dp"
        android:textStyle="bold"/>

    <LinearLayout
        android:id="@+id/start_button_layout"
        android:layout_width="300dp"
        android:layout_height="150dp"
        android:layout_gravity="center"
        android:background="@drawable/start_button_ripple"
        android:gravity="center">

        <TextView
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="@string/start"
            android:textColor="?colorOnSecondary"
            android:textStyle="bold"
            android:textSize="45sp"/>

    </LinearLayout>

</FrameLayout>
```

I na koniec zmieńmy tytuł pojawiający się na `ActionBar` oraz dodajmy `onClick`

In [ ]:
class StartFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        val activity: AppCompatActivity = activity as AppCompatActivity
        activity.supportActionBar?.title = getString(R.string.app_name)
        return inflater.inflate(R.layout.fragment_start, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        view.findViewById<LinearLayout>(R.id.start_button_layout).setOnClickListener {
            findNavController().navigate(StartFragmentDirections.actionStartFragmentToModuleListFragment())
        }
    }
}

<img src="https://media4.giphy.com/media/vxFCJk2ezJecGe0q51/giphy.gif?cid=790b7611375b0d8e309f01295b707e9b33cd918c6b2cec75&rid=giphy.gif&ct=g" width="150" />

### **Dane**

Zacznijmy się teraz przygotowaniem danych do naszej aplikacji. Na tym etapie nie będziemy jeszcze wykorzystywać żadnej bazy danych ani backendu, więc wszystko wrzucimy hardcoded.

In [ ]:
data class App(
    val id: Int,
    val apps: List<String>
)

data class Lab(
    val id: Int,
    val name: String,
    val content: List<String>
)

data class Lecture (
    val id: Int,
    val name: String,
    val content: List<String>
)

data class Module(
    val id: Int,
    val name: String,
    val lecture: Lecture,
    val lab: Lab,
    val apps: App
)

In [ ]:
object DataProvider {
    private val lectures = listOf(
        Lecture(0, "Podstawowe Informacje", listOf("podstawowe informacje", "treści programowe", "warunki zaliczenia")),
        Lecture(1, "Cykl życia aktywności", listOf("Struktura projektu", "Activity", "Cykl życia aktywności", "Stan instancji", "Bundle", "Intent")),
        Lecture(2, "RecyclerView", listOf("RecyclerView", "Adapter", "ViewHolder", "Selector")),
        Lecture(3, "Fragmenty", listOf("Jetpack Navigation", "Navigation Drawer", "Fragment"))
    )

    private val labs = listOf(
        Lab(0, "Przygotowanie layoutu", listOf("LinearLayout", "ConstraintLayout", "Obsługa onClick")),
        Lab(1, "Mechanizm intentów", listOf("Mechanizm intentów - jawne", "Intefejsy Serializable, Parcelable", "Mechanizm intentów - domniemane")),
        Lab(2, "RecyclerView", listOf("RecyclerView", "Selector")),
        Lab(3, "Nawigacja", listOf("Fragmenty statyczne", "Fragmenty dynamiczne", "Navigation Drawer", "RecyclerView"))
    )

    private val apps = listOf(
        App(0, listOf("CounterApp")),
        App(1, listOf("QuizApp")),
        App(2, listOf("WFiApp", "QuickYoga")),
        App(3, listOf("PUMApp"))
    )

    val modules: List<Module>
        get() = (0 until 4).map { Module(
            it,
            "Moduł $it",
            lectures[it],
            labs[it],
            apps[it]
        ) }
}

### **ModuleListFragment**

Listę wszystkich modułów będziemy wyświetlać przez `RecyclerView`. Rozpocznijmy od przygotowania layoutu pojedynczego elementu - wyświetlimy nazwę modułu oraz tytuł wykładu

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="72dp"
    android:layout_margin="24dp"
    app:cardBackgroundColor="#0b5294"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:orientation="vertical">

        <TextView
            android:id="@+id/moduleNameModuleRVItemTextView"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_gravity="center_horizontal"
            android:text="@string/module"
            android:textColor="@color/white"
            android:textSize="26sp" />

        <TextView
            android:id="@+id/lectureNameModuleRVItemTextView"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginStart="16dp"
            android:text="@string/lecture_long_name"
            android:textColor="@color/white"
            android:textSize="18sp" />


    </LinearLayout>
</androidx.cardview.widget.CardView>
```

ponieważ wykorzystujemy `CardView` dodajmy odpowiednią zależność do `build.Gradle(Module)`

```java
implementation("androidx.cardview:cardview:1.0.0")
```

Następnie dodajmy `RecyclerView` do layoutu `ModuleListFragment`

```xml
<?xml version="1.0" encoding="utf-8"?>
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.ModuleListFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/moduleRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:clipToPadding="false"
        android:padding="16dp" />

</FrameLayout>
```

Następnie napiszmy `Adapter`

In [ ]:
class ModuleListAdapter : RecyclerView.Adapter<ModuleListAdapter.ViewHolder>() {

    private val moduleList = DataProvider.modules

    inner class ViewHolder(view: View) : RecyclerView.ViewHolder(view) {
        private val moduleTitleTextView: TextView = view.findViewById(R.id.moduleNameModuleRVItemTextView)
        private val lectureTitleTextView: TextView = view.findViewById(R.id.lectureNameModuleRVItemTextView)

        fun bind(item: Module){
            moduleTitleTextView.text = item.name
            lectureTitleTextView.text = item.lecture.name
        }
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ViewHolder {
        return ViewHolder(LayoutInflater
            .from(parent.context)
            .inflate(R.layout.module_list_item_view, parent, false))
    }

    override fun onBindViewHolder(holder: ViewHolder, position: Int) {
        val item = moduleList[position]
        holder.bind(item)
    }

    override fun getItemCount(): Int = moduleList.size
}

Ostatnim elementem będzie modyfikacja klasy `ModuleListFragment`

In [ ]:
class ModuleListFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        val activity: AppCompatActivity = activity as AppCompatActivity
        activity.supportActionBar?.title = getString(R.string.module_list)
        return inflater.inflate(R.layout.fragment_module_list, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        val recyclerView = view.findViewById<RecyclerView>(R.id.moduleRecyclerView)
        recyclerView.apply {
            layoutManager = LinearLayoutManager(requireContext())
            adapter = ModuleListAdapter()
        }
    }
}

### **ModuleFragment**

Dodajmy `onClick` do `ModuleListAdapter` oraz `<argument>` do nawigacji umożliwiający przesłanie `id` klikniętego modułu. Rozpocznijmy od argumentu - przechodzimy do pliku `navigation.xml` i do `moduleListFragment` dodajemy argument typu `int`

```xml
<fragment
    android:id="@+id/moduleListFragment"
    android:name="pl.edu.uwr.pum.pumappjava.fragments.ModuleListFragment"
    android:label="fragment_module_list"
    tools:layout="@layout/fragment_module_list" >
    <action
        android:id="@+id/action_moduleListFragment_to_moduleFragment"
        app:destination="@id/moduleFragment" >
        <argument
            android:name="moduleId"
            app:argType="integer" />
    </action>
</fragment>
```

Następnie zdefiniujmy metodę `onClick` w `ModuleListAdapter`

In [ ]:
override fun onBindViewHolder(holder: ViewHolder, position: Int) {
    val item = moduleList[position]
    holder.bind(item)
    holder.itemView.setOnClickListener {
        holder.itemView.findNavController().navigate(
            ModuleListFragmentDirections.actionModuleListFragmentToModuleFragment(
                moduleId = item.id
            )
        )
    }
}

Przejdźmy do `ModuleFragment` - potrzebujemy dostać `Module` o otrzymanym `id`. Zdefiniujmy pole `module`

In [ ]:
private val module by lazy {
    val moduleId = arguments?.getInt("moduleId")?:
        throw IllegalArgumentException("moduleId doesn't exist")
    DataProvider.modules[moduleId]
}

Zmodyfikujmy layout `ModuleFragment` - będziemy potrzebować trzy przyciski, przez które będziemy przenosić się do elementów modułu (wykład, lab, aplikacje)

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".fragments.ModuleFragment">

    <Button
        android:id="@+id/lectureButton"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="16dp" />

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:baselineAligned="false"
        android:orientation="horizontal">


        <Button
            android:id="@+id/labButton"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_margin="16dp"
            style="@style/Theme.PUMAppJava"
            android:textAlignment="center"
            android:layout_height="72dp"
            android:layout_marginStart="16dp" />

        <Button
            android:id="@+id/appButton"
            android:layout_width="0dp"
            android:layout_weight="1"
            style="@style/Theme.PUMAppJava"
            android:textAlignment="center"
            android:layout_margin="16dp"
            android:layout_height="72dp" />

    </LinearLayout>

</LinearLayout>
```

Tekst przycisków będzie odpowiadał tytułowi elementu. Wpierw zmodyfikujmy tytuł fragmentu. Przechodzimy do metody `onCreateView`

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View? {
    val activity: AppCompatActivity = activity as AppCompatActivity
    activity.supportActionBar?.title = module.name
    return inflater.inflate(R.layout.fragment_module, container, false)
}

Tytuł będzie odpowiadał tytułowi modułu. Następnie, w metodzie `onViewCreated`, ustawiamy tekst przycisków.

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    view.findViewById<Button>(R.id.lectureButton).apply { 
        text = "Wykład:\n${module.lecture.name}"
    }

    view.findViewById<Button>(R.id.labButton).apply { 
        text = "Laboratorium:\n${module.lab.name}"
    }

    view.findViewById<Button>(R.id.appButton).apply { 
        text = requireContext().getString(R.string.aplications)
    }
}

Ostatnim elementem będzie dodanie obsługi `onClick` oraz argumentów do akcji. Przejdźmy do `navigation.xml` i zmodyfikujmy `moduleFragment`

```xml
<fragment
    android:id="@+id/moduleFragment"
    android:name="pl.edu.uwr.pum.pumappjava.fragments.ModuleFragment"
    android:label="fragment_module"
    tools:layout="@layout/fragment_module" >
    <action
        android:id="@+id/action_moduleFragment_to_lectureFragment"
        app:destination="@id/lectureFragment" >
        <argument
            android:name="moduleId"
            app:argType="integer" />
    </action>
    <action
        android:id="@+id/action_moduleFragment_to_labListFragment"
        app:destination="@id/labListFragment" >
        <argument
            android:name="moduleId"
            app:argType="integer" />
    </action>
    <action
        android:id="@+id/action_moduleFragment_to_appListFragment"
        app:destination="@id/appListFragment" >
        <argument
            android:name="moduleId"
            app:argType="integer" />
    </action>
</fragment>
```

Przy każdej zmianie ekranu będziemy przekazywać `mopduleId` dzięki któremu wyciągniemy odpowiednie dane z `DataProvider`. Dodajmy metodzy `onClick` naszych przycisków do metody `onViewCreated`

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    view.findViewById<Button>(R.id.lectureButton).apply { 
        text = "Wykład:\n${module.lecture.name}"
        setOnClickListener {
            findNavController().navigate(ModuleFragmentDirections.actionModuleFragmentToLectureFragment(
                moduleId = module.id
            ))
        }
    }

    view.findViewById<Button>(R.id.labButton).apply { 
        text = "Laboratorium:\n${module.lab.name}"
        setOnClickListener {
            findNavController().navigate(ModuleFragmentDirections.actionModuleFragmentToLabListFragment(
                moduleId = module.id
            ))
        }
    }

    view.findViewById<Button>(R.id.appButton).apply { 
        text = requireContext().getString(R.string.aplications)
        setOnClickListener {
            findNavController().navigate(ModuleFragmentDirections.actionModuleFragmentToAppListFragment(
                moduleId = module.id
            ))
        }
    }
}

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/a0efhMDjbTeqZJl3oO/giphy.gif?cid=790b7611970400a9fec62348cfb0475e5a6876fd87c51c83&rid=giphy.gif&ct=g" width="150" />